<a href="https://colab.research.google.com/github/leonelmoreno-cmd/Amazon_Finder/blob/main/Snovio_token.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Los emails devueltos están sin verificar; si quieres validarlos, usa el Email Verifier de Snov.io.
#The API rate is limited to 60 requests per minute.

import requests
import json
import time

def get_access_token():
    url = "https://api.snov.io/v1/oauth/access_token"
    params = {
        "grant_type": "client_credentials",
        "client_id": "fa6be1cc94a9a3a611be449f943be42d",
        "client_secret": "195ef22a0be658fa654f9cf0f893d8da"
    }

    response = requests.post(url, data=params)
    data = response.json()  # convierte la respuesta JSON en un diccionario
    return data["access_token"]

In [2]:
token = get_access_token()

In [3]:
def get_domain_email_count(domain, token):
    """
    Llamada GRATUITA: te dice cuántos emails existen para el dominio
    y si es webmail (gmail, yahoo...). Si webmail=True, siempre 0.
    Útil para decidir si vale la pena gastar créditos luego.
    """
    url = "https://api.snov.io/v1/get-domain-emails-count"
    data = {"access_token": token, "domain": domain}
    r = requests.post(url, data=data, timeout=30)
    r.raise_for_status()
    return r.json()  # {success, domain, webmail, result}


In [4]:
def start_domain_emails(domain, token):
    """
    Inicia una 'tarea' que extrae emails de un dominio.
    - Primera página: manda domain.
    - Páginas siguientes: manda 'next' (te lo da la respuesta anterior).
    Devuelve:
      - task_hash (ID de la tarea)
      - result_url (endpoint GET para consultar el resultado)
    """
    url = "https://api.snov.io/v2/domain-search/domain-emails/start"
    headers = {"Authorization": f"Bearer {token}"}
    params = {"domain": domain}

    r = requests.post(url, params=params, headers=headers, timeout=30)
    r.raise_for_status()
    payload = r.json()
    task_hash = payload["meta"]["task_hash"]
    result_url = payload["links"]["result"]
    return task_hash, result_url


In [5]:
def poll_domain_emails_result(task_hash, token, poll_every=3, max_wait=60):
    """
    Consulta periódicamente el resultado de la tarea:
      - Si status == 'in progress', espera y vuelve a consultar.
      - Si status == 'completed', devuelve:
          * emails (lista de emails de esa página)
          * next_link (URL para pedir la siguiente página; puede estar vacío)
          * meta (info útil: total_count, etc.)
    """
    url = f"https://api.snov.io/v2/domain-search/domain-emails/result/{task_hash}"
    headers = {"Authorization": f"Bearer {token}"}
    waited = 0

    while waited <= max_wait:
        r = requests.get(url, headers=headers, timeout=30)
        r.raise_for_status()
        resp = r.json()
        status = resp.get("status")  # "completed" | "in progress"

        if status == "completed":
            emails = [item["email"] for item in resp.get("data", [])]
            next_link = resp.get("links", {}).get("next")
            meta = resp.get("meta", {})
            return emails, next_link, meta

        time.sleep(poll_every)
        waited += poll_every

    raise TimeoutError("La tarea tardó demasiado en completarse")


In [6]:
def fetch_emails_first_page(domain, token):
    """
    Obtiene SOLO la primera página de correos de un dominio desde Snov.io.

    Pasos:
    1. Inicia la búsqueda del dominio.
    2. Espera hasta que la API complete la tarea.
    3. Devuelve los correos encontrados (sin paginar).
    """
    all_emails = []

    # 1️⃣ Inicia la búsqueda
    task_hash, _ = start_domain_emails(domain, token)

    # 2️⃣ Espera el resultado de esa búsqueda
    emails, next_link, meta = poll_domain_emails_result(task_hash, token)

    # 3️⃣ Guarda los correos encontrados
    all_emails.extend(emails)

    # 4️⃣ Devuelve lista de correos (sin duplicados) y los metadatos
    return list(dict.fromkeys(all_emails)), meta


In [7]:
def fetch_emails_for_domains(domains, token, skip_webmail=True):
    """
    Recorre tus dominios y aplica el flujo:
      - (Gratis) get_domain_email_count: si es webmail o 0 resultados, evita gastar créditos.
      - Si hay resultados, saca todas las páginas con fetch_emails_first_page.
    Devuelve un dict:
      {
        "dominio.com": {
          "emails": [...],
          "total_count_reported": 108,
          "note": "..."
        },
        ...
      }
    """
    results = {}

    for domain in domains:
        try:
            # Paso opcional y gratis para filtrar
            cnt = get_domain_email_count(domain, token)

            if skip_webmail and cnt.get("webmail"):
                results[domain] = {"emails": [], "note": "Dominio webmail; Snov devuelve 0"}
                continue

            if cnt.get("result", 0) == 0:
                results[domain] = {"emails": [], "note": "Sin emails en la base (0)"}
                continue

            emails, meta = fetch_emails_first_page(domain, token)
            results[domain] = {
                "emails": emails,
                "total_count_reported": meta.get("total_count"),
                "note": "Unverified emails; use Email Verifier to validate them."
            }

        except Exception as e:
            # Captura errores por dominio para no abortar todo el lote
            results[domain] = {"emails": [], "error": str(e)}

    return results


In [8]:
dominios = ["www.affresh.com"]
resultado = fetch_emails_for_domains(dominios, token)
print(json.dumps(resultado, indent=2, ensure_ascii=False))


{
  "www.affresh.com": {
    "emails": [
      "info@affresh.com",
      "johnsmith@affresh.com",
      "john.smith@affresh.com"
    ],
    "total_count_reported": 3,
    "note": "Emails sin verificar; usa Email Verifier para validarlos."
  }
}
